If you do not have the label file, run this cell.

In [ ]:
!wget https://raw.githubusercontent.com/tproffen/SmartRobotCamp/master/Notebooks/Jetbot/ObjectFollow/mscoco_complete_label_map.json

## Detect Objects

We'll use a pre-trained neural network
that was trained on the [COCO dataset](http://cocodataset.org) to detect 90 different common objects - you can check [this file](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt) for a full list of class indices). First, we'll want to import the ``ObjectDetector`` class which takes our pre-trained SSD engine.

In [ ]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

Create a camera instance.

In [ ]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg

camera = Camera.instance(width=512, height=512)

Widget for the image. This will show up when we link the camera to the widget later. You might want to create a separate view for the camera feed.

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets

image_widget = widgets.Image(format='jpeg', width=512, height=512)
display(image_widget)

The network only gives us an index. Here we create a list to look up the label we want to display.

In [ ]:
import json

descriptions=[]

with open('mscoco_complete_label_map.json') as json_file:
    data = json.load(json_file)
    for p in data['items']:
        descriptions.append(p['display_name'])

The code below has functions to draw a box and a label around the detected objects. It also contains the routine execute that is called every frame later on. 

In [ ]:
import cv2
import numpy as np

width = int(image_widget.width)
height = int(image_widget.height)

def drawBoundingBox(imgcv,box,label,color):
    x1,y1,x2,y2 = (int(width * box[0]), int(height * box[1]), int(width * box[2]), int(height * box[3]))
    cv2.rectangle(imgcv,(x1,y1),(x2,y2),color,2)
    labelSize=cv2.getTextSize(label,cv2.FONT_HERSHEY_COMPLEX,0.5,2)
    _x1 = x1
    _y1 = y1
    _x2 = x1+labelSize[0][0]
    _y2 = y1-int(labelSize[0][1])
    cv2.rectangle(imgcv,(_x1,_y1),(_x2,_y2),color,cv2.FILLED)
    cv2.putText(imgcv,label,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1)
    return imgcv
    

def execute(change):
    image = change['new']
    
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    
    for det in detections[0]:
        image=drawBoundingBox(image,det['bbox'],descriptions[det['label']],(255,0,0))

    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

The cell above runs this detection once. Run the cell below to keep running it everytime the camera image changes.

In [ ]:
camera.unobserve_all()
camera.observe(execute, names='value')

Finally run this cell to stop the object detection :)

In [ ]:
camera.unobserve_all()